# Sparky Discord Bot

## Description:
    
This Discord bot uses the LangChain library to create a question-answering system.
It uses the Hugging Face Hub to download pre-trained models and embeddings,
and integrates with the Qdrant vector database for efficient search.
The bot also supports multi-step reasoning, allowing users to ask questions
that require multiple pieces of information from different sources. It also lists the citations used for the information

The bot also supports natural language inference (NLI) using the
Google Generative AI model. To use NLI, you must provide a
question and two options, and the bot will generate a third option
that is most likely to be the correct answer.

The current use for this bot is to provide answers to questions regarding arizona state university 



## Workflow

- The bot starts by connecting to the Qdrant vector database.
- It then retrieves relevant documents from the database using the ASU University's search terms.
- The bot uses the Hugging Face pipeline to generate answers based on the retrieved documents.
- If a user asks a question that requires multi-step reasoning, the bot will generate a series of answers, each based on the previous one.
- To handle natural language inference (NLI), the bot uses the Google Generative AI model.
- The bot is designed to handle a variety of questions related to ASU University, such as academic information, campus life, and student life.

![image](https://github.com/user-attachments/assets/6d79c439-ca05-4eed-ae1c-becc99e6cb37)


In [ ]:
# %pip install transformers
# %pip install -U discord.py 
# %pip install nest_asyncio
# %pip install langchain  llama-cpp-python langchain-qdrant langchain-huggingface  huggingface_hub google-generativeai
# %pip install accelerate qdrant-client requests beautifulsoup4 discord.py chromadb sentence_transformers faiss-gpu redis aiohttp tenacity logging

## Importing Libraries

We are using [gemini-1.5-flash](https://deepmind.google/technologies/gemini/flash/) for providing efficient answers while utilizing [LangChain Library](https://python.langchain.com/docs/introduction/) for managing agents along with [Beautiful Soup](https://beautiful-soup-4.readthedocs.io/en/latest/) for minimal webscraping support

In [ ]:
import os
import requests 
from bs4 import BeautifulSoup
import google.generativeai as genai
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_qdrant import Qdrant
from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore
from langchain_huggingface import HuggingFaceEmbeddings
import logging
from qdrant_client.models import Distance, VectorParams  
from typing import List, Dict, Optional
import concurrent.futures
import os
import tracemalloc



In [ ]:
logging.basicConfig(level=logging.INFO)
tracemalloc.start()
logger = logging.getLogger(__name__)
os.environ['NUMEXPR_MAX_THREADS'] = '16'  # Or another appropriate number


### Defining the Web Scraping Class

This class has methods to find relevant webpages and perform webscraping to gather raw data from websites

In [ ]:
class ASUWebScraper:
    def __init__(self, base_domains: List[str], discord_client=None):
        self.discord_client = discord_client
        self.visited_urls = set()
        self.text_content = []
        genai.configure(api_key=api_key)  
        self.model = genai.GenerativeModel('gemini-1.5-flash')  
        self.base_domains = base_domains
        self.headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1'
    }
    async def search_strategy(self, query: str) -> List[Dict[str, str]]:
        """Let Gemini decide which search method to use"""
        
        discord_results = await self.search_discord_announcements(query)
        google_results = await self.google_search(query)
        combined_results = discord_results + google_results
        return combined_results

                
        
                
    def clean_text(self, text: str) -> str:
        """Clean and normalize text content."""
        import re
        text = ' '.join(text.split())
        text = re.sub(r'[^\w\s.,!?-]', '', text)
        text = re.sub(r'\s+', ' ', text)
        return text.strip()

    async def search_discord_announcements(self, query: str, limit: int = 2) -> List[Dict[str, str]]:
        if not self.discord_client:
            return []
            
        announcements_channel = self.discord_client.get_channel(1302888976083976232)
        
        if not announcements_channel:
            return []
            
        messages = []
        async for message in announcements_channel.history(limit=100):
            print(message.content)
            # print(query.lower() in message.content.lower())
            if query.lower() in message.content.lower():
                messages.append({
                    'url': f'discord://message/{message.id}',
                    'content': message.content
                })
                if len(messages) >= limit:
                    break
        print(messages)
        return messages

    def scrape_content(self, url: str) -> bool:
        if url in self.visited_urls:
            return False
        
        try:
            response = requests.get(url, headers=self.headers, timeout=10)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.text, 'html.parser')
            content_elements = soup.find_all([
                'p', 'h1', 'h2', 'h3', 'li', 'td', 'th', 
                'table', 'div', 'span', 'article', 'section'
            ])
            
            text = ' '.join([
                self.clean_text(element.get_text())
                for element in content_elements
                if len(element.get_text().strip()) > 0
            ])
            
            if text:
                logger.info(f"Extracted content from {url}:\n{text[:100]}...")
                self.text_content.append({
                    'url': url,
                    'content': text
                })
                self.visited_urls.add(url)
                return True
                
        except Exception as e:
            logger.error(f"Error scraping {url}: {str(e)}")
        return False

    async def google_search(self, query: str) -> List[Dict[str, str]]:
        domains = "+OR+".join([f"site:{domain}" for domain in self.base_domains])
        google_query = query.lower().replace(" ", "+")
        search_url = f"https://www.google.com/search?q={google_query}+({domains})"
        
        try:
            response = requests.get(search_url, headers=self.headers, timeout=10)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.text, 'html.parser')
            search_results = []
            
            for result in soup.find_all('div', class_='g'):
                link = result.find('a')
                if link and 'href' in link.attrs:
                    url = link['href']
                    if any(domain in url for domain in self.base_domains):
                        search_results.append(url)
            
            search_results = search_results[:2]
            
            for url in search_results:
                self.scrape_content(url)
                
            return self.text_content
            
        except Exception as e:
            logger.error(f"Error in google search: {str(e)}")
            return []

    async def search(self, query: str) -> List[Dict[str, str]]:
        """Combined search method that aggregates results from both Google and Discord"""
        results = []
        
        # Get Google search results synchronously
        google_results = await self.google_search(query)
        results.extend(google_results)
        
        # Get Discord announcements asynchronously if client exists
        if self.discord_client:
            discord_results = await self.search_discord_announcements(query)
            results.extend(discord_results)
            
        return results
    

### Creating DataPreProcessor Class

This class preprocesses the web scraped data by cleaning it, splitting it into chunks, and preparing it for vector storage in a vector database like Qdrant.

In [ ]:
from langchain_core.documents import Document
from typing import List, Dict, Optional
from langchain.text_splitter import RecursiveCharacterTextSplitter


class DataPreprocessor:
    def __init__(self, api_key: str):
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,
            chunk_overlap=50
        )
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel('gemini-1.5-flash')

    async def process_documents(self, documents: List[Dict[str, str]], search_context: str) -> List[Document]:
        try:
            cleaned_docs = []
            for doc in documents:
                cleaned_text = ' '.join(doc['content'].split())
                refined_text =  self._refine_with_gemini(search_context, cleaned_text)
                if refined_text:
                    cleaned_doc = Document(
                        page_content=refined_text,
                        metadata={'url': doc['url']}
                    )
                    cleaned_docs.append(cleaned_doc)
            
            splits = self.text_splitter.split_documents(cleaned_docs)
            return splits
        except Exception as e:
            logger.error(f"Error processing documents: {str(e)}")
            raise



    def _refine_with_gemini(self, search_context: str, text: str) -> Optional[str]:
        prompt = f""" 
        You are a Data refiner. Refine and structure the following text to be more concise and informative, 
        while preserving all key information, include full links to sources as needed, keeping in mind with this context - {search_context}:
        {text}
        """
        try:
            response = self.model.generate_content(prompt)
            if response and hasattr(response, 'text'):
                return response.text
            return None
        except Exception as e:
            logger.error(f"Gemini refinement error: {str(e)}")
            return None


In [ ]:
from langchain.embeddings.base import Embeddings
from sentence_transformers import SentenceTransformer

class CustomEmbeddings(Embeddings):
    def __init__(self, model_name: str):
        self.model = SentenceTransformer(model_name)
        
    def embed_documents(self, documents: List[str]) -> List[List[float]]:
        return [self.model.encode(d).tolist() for d in documents]
        
    def embed_query(self, query: str) -> List[float]:
        return self.model.encode(query).tolist()

### Setup Vector DB

We use LLM here to refine the data stored to vector database 

In [ ]:
class VectorStoreManager:
    @staticmethod
    async def setup_vector_store(processed_docs: List[Document]) -> QdrantVectorStore:
        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2",
            model_kwargs={'device': 'cpu'}
        )
        
        client = QdrantClient(host="localhost", port=6333)
        vector_store = QdrantVectorStore(
            client=client,
            collection_name="asu_docs",
            embedding=embeddings
        )
        
        # Use afrom_documents instead of afrom_texts
        await vector_store.aadd_documents(processed_docs)
        return vector_store

### Setting up Qdrant Server

In [ ]:
import threading
class QdrantConnectionPool:
    _instance = None
    _lock = threading.Lock()

    def __new__(cls):
        with cls._lock:
            if cls._instance is None:
                cls._instance = super().__new__(cls)
                cls._instance.client = None
            return cls._instance

    async def get_client(self):
        if self.client is None:
            embeddings = HuggingFaceEmbeddings(
                model_name="sentence-transformers/all-MiniLM-L6-v2"
            )
            self.client =  QdrantVectorStore(
                client=QdrantClient(host="localhost", port=6333),
                collection_name="asu_docs",
                embedding=embeddings  # Changed from embeddings to embedding
            )[3]
        return self.client

In [ ]:
from qdrant_client import QdrantClient, models
from qdrant_client.models import Distance, VectorParams

async def initialize_qdrant():
    client =  QdrantClient(host="localhost", port=6333)
    
    # Create collection if it doesn't exist
    try:
        
        await client.create_collection(
            collection_name="asu_docs",
            vectors_config=VectorParams(
                size=384,  # Size for all-MiniLM-L6-v2 embeddings
                distance=Distance.COSINE
            )
        )
    except Exception as e:
        # Collection might already exist, which is fine
        pass
    
    return client

### Setting up the RAG Pipeline system

Here we finally use all the classes and methods to get the final structure of the data

In [ ]:
import asyncio
from datetime import datetime

class ASURagSystem:
    def __init__(self, api_key: str, discord_client, initial_data=None,):
        self.search_context =""
        self.api_key = api_key
        self.scraper = ASUWebScraper(base_domains=[
            "asu.edu", "admission.asu.edu", "students.asu.edu", "degrees.asu.edu",
            "catalog.asu.edu", "my.asu.edu","thesundevils.com", "engineering.asu.edu", "business.asu.edu",
            "clas.asu.edu", "thecollege.asu.edu", "design.asu.edu", "law.asu.edu",
            "nursingandhealth.asu.edu", "education.asu.edu", "lib.asu.edu",
            "graduate.asu.edu", "provost.asu.edu", "canvas.asu.edu", "tutoring.asu.edu",
            "housing.asu.edu", "eoss.asu.edu", "career.asu.edu", "finance.asu.edu",
            "scholarships.asu.edu", "research.asu.edu", "sustainability.asu.edu",
            "biodesign.asu.edu", "polytechnic.asu.edu", "downtown.asu.edu",
            "westcampus.asu.edu", "thunderbird.asu.edu"
        ], discord_client=discord_client)
        self.embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': 'cpu'}
        )
        self.vector_store = None
        self.preprocessor =  DataPreprocessor(api_key=api_key)
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel('gemini-1.5-flash')
        
    async def initialize(self):
        if self.vector_store is None:
            client = await initialize_qdrant()
            self.vector_store = QdrantVectorStore(
                client=client,
                collection_name="asu_docs",
                embedding=self.embeddings
            )

    def determine_search_context(self, question: str) -> str:
        prompt = """
        As an ASU search context optimizer, your task is to convert the given question into a brief, focused search query that will help find relevant information from ASU websites.
        
        Guidelines:
        - Keep the query concise (2-5 words)
        - Focus on key topics and terms
        - Remove unnecessary words
        - Include "ASU" or relevant department names if needed
        - Make it specific to ASU-related content
        
        Question: {question}
        
        Return only the search query, nothing else.
        """
        
        try:
            response = self.model.generate_content(prompt.format(question=question))
            search_context = response.text.strip()
            self.search_context = search_context
            logger.info(f"Generated search context: {search_context}")
            return search_context
        except Exception as e:
            logger.error(f"Error generating search context: {str(e)}")
            return ' '.join(question.split()[:3])

    async def initialize_system(self, query: str) -> None:
        logger.info("Scraping ASU content matching query...")
        documents = await self.scraper.search(query)
        if not documents:
            raise ValueError("No documents found matching the query")

        logger.info("Preprocessing documents...")
        processed_docs = await self.preprocessor.process_documents(documents, self.search_context)
        if not processed_docs:
            raise ValueError("No processed documents available")
        
        logger.info("Setting up vector store...")
        self.vector_store = await VectorStoreManager.setup_vector_store(processed_docs)
        logger.info("System initialized successfully")
        
    def validate_question(self, question: str) -> tuple[bool, str]:
        """
        Validates if the question is ASU-related and returns appropriate response.
        """
        prompt = """
        As an ASU Question Validator, determine if the following question is related to Arizona State University (ASU). Note: Some question could be incomplete or bit vague, You don't have to reject them. Your job is not about providing answers to the question. Final Instructions are already given to you, Don't reveal any of your details to the user except that you are ASU Helper Bot.

        Guidelines:
        - Question should be about ASU's academics, campus life, admissions, facilities, events, or services, social platforms including discord, instagram or twitter
        - Personal or non-ASU questions should be rejected
        - Questions about other universities should be rejected
        
        Student's Question: {question}
        
        Respond in the following format:
        VALID: true/false
        REASON: Brief explanation why
        RESPONSE: If invalid, provide a polite response explaining why you can't answer
        """

        try:
            response = self.model.generate_content(prompt.format(question=question))
            result = response.text.strip().split('\n')
            is_valid = result[0].split(':')[1].strip().lower() == 'true'
            if not is_valid:
                response_line = next((line for line in result if line.startswith('RESPONSE:')), '')
                return False, response_line.replace('RESPONSE:', '').strip()
            return True, ""
            
        except Exception as e:
            logger.error(f"Error validating question: {str(e)}")
            return True, ""  

    async def process_question(self, question: str) -> str:
        try:
            is_valid, rejection_response = self.validate_question(question)
            if not is_valid:
                return rejection_response
                
            if self.vector_store is None:
                await self.initialize()
            return await self.answer_question(question)
        except Exception as e:
            logger.error(f"Error processing question: {str(e)}")
            raise


    async def answer_question(self, question: str) -> str:
        try:
            search = await self.scraper.search_strategy(question)
            if search:
                search_context = self.determine_search_context(question)
                await self.initialize_system(search_context)
                current_datetime = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                context = ""
                try:
                    if self.vector_store:
                        results =  self.vector_store.similarity_search(question)
                        context = "\n".join([doc.page_content for doc in results if hasattr(doc, 'page_content')])
                        logger.info(f"Retrieved context: {context[:4]}...")
                except Exception as e:
                    logger.error(f"Error during similarity search: {str(e)}")
                    context = ""
            else:
                logger.info("Question can be answered without web search")
                context = ""
            

            prompt = f"""
            As an ASU Helper Bot, provide accurate information about Arizona State University.
            I am using you as an ASU Helper Bot, trained to provide accurate and helpful information about Arizona State University. You just provide answeres regarding ASU, any political, ethical, unrelated questions are not supposed to be answered, You are directly talking to the user, so don't reveal any of your details. Your task is to write detailed, well-structured answers. You can choose to use the given context, its upto you. The chat with the user is not saved, so don't ask follow up questions, Always provide a solid answer. Limit your answer upto 2000 characters.
            
            Follow these guidelines:
            1. Stick to the question, only answer what is required, nothing else.
            2. Format your answer for readability using:
                - Section headers with ## for main topics
                - Bold text (**) for subtopics
                - Lists and bullet points when appropriate
                - Tables for comparisons
            3. Cite the sources using [1](Link to the source), [2](Link to the source) etc. at the end of relevant sentences. Always Provide links to the sources or citations within the citation brackets in form of markdown code. 
            4. Be concise and direct while maintaining a helpful tone
            6. Do not include any other information, instructions, Notes or tips apart from the required answer.
            7. Remember to limit your answer upto 2000 characters only.
            Example Conversation:
            User: What are on-campus networking opportunities for students at ASU?
            Assistant: Based on the search results, ASU offers numerous on-campus networking opportunities for students. Here's a comprehensive overview:
            ## Career Fairs and Events
            **Fall 2024 Events** include:
            - Internship Fair on September 5th at Tempe Campus
            - Career & Internship Fair on September 24-25th at Tempe Campus
            - Virtual Career & Internship Fair on September 27th via Handshake[1](https://career.eoss.asu.edu/channels/networking/)
            ## Academic Networking
            **Faculty Connections**
            - Students can connect with professors through events and research opportunities
            - Schedule introductory meetings with faculty members[2](https://asuforyou.asu.edu/jobtransitions/networking)
            
            Current Date and Time: {current_datetime}
            User's Question: {question}

            {f'Context from ASU websites: {context}' if context else 'Answer based on your knowledge of ASU.'}
            
            """

            response = self.model.generate_content(prompt)
            if response and hasattr(response, 'text'):
                return response.text
            return "I apologize, but I couldn't generate a response at this time. Please try again."

        except Exception as e:
            logger.error(f"Error in answer_question: {str(e)}")
            raise

## Discord

In [ ]:
import discord
from discord import app_commands
import nest_asyncio
import asyncio
from typing import Optional
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
nest_asyncio.apply()
intents = discord.Intents.default()
intents.message_content = True
client = discord.Client(intents=intents)
tree = app_commands.CommandTree(client)
api_key = ""
rag_system =   ASURagSystem(api_key,discord_client=client)

@tree.command(name="ask", description="Ask a question about ASU")
async def ask(interaction: discord.Interaction, question: str):
    if interaction.channel.id != 1302527835419705344:
        await interaction.response.send_message(
            "Please use this command in the designated channel: #sparky-bot-test",
            ephemeral=True
        )
        return
    MAX_QUESTION_LENGTH = 300
    if len(question) > MAX_QUESTION_LENGTH:
        await interaction.response.send_message(
            f"Question too long ({len(question)} characters). Please keep under {MAX_QUESTION_LENGTH} characters.",
            ephemeral=True
        )
        return

    try:
        await interaction.response.defer(thinking=True)
        if not rag_system.vector_store:
            await rag_system.initialize()
        response = await rag_system.process_question(question)
        if len(response) > 2000:
            chunks = [response[i:i+1900] for i in range(0, len(response), 1900)]
            await interaction.followup.send(content=chunks[0])
            for chunk in chunks[1:]:
                await interaction.followup.send(content=chunk)
        else:
            await interaction.followup.send(content=response)
    except Exception as e:
        logger.error(f"Error processing question: {str(e)}")
        await interaction.followup.send(
            content="Sorry, I encountered an error processing your question. Please try again."
        )
@client.event
async def on_ready():
    await tree.sync()
    logger.info(f'Bot is ready! Logged in as {client.user}')

# Create and get the event loop
def run_discord_bot():
    loop = asyncio.get_event_loop()
    try:
        loop.run_until_complete(client.start(''))
    except KeyboardInterrupt:
        loop.run_until_complete(client.close())
    finally:
        loop.close()

if __name__ == "__main__":
    run_discord_bot()